In [ ]:
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import matplotlib.pyplot as plt

# Compare the two target values

In [ ]:
df = pd.read_csv('data/qm8.csv')

In [ ]:
df2 = pd.read_csv('data/qm8.sdf.csv')

In [ ]:
df.columns

In [ ]:
df2.columns

In [ ]:
target_cols = ['E1-CC2', 'E2-CC2', 'f1-CC2', 'f2-CC2']

In [ ]:
for target in target_cols:
    print("Checking", target)
    print(np.allclose(df[target].values, df2[target].values))

In [ ]:
def check_3d(pos):
    return not np.any(np.all(np.isclose(pos, pos[0]), 0))

# Loading the dataset

In [ ]:
suppl = Chem.SDMolSupplier('data/qm8.sdf', removeHs=False, sanitize=False)

In [ ]:
Zs, euclid_Ds, graph_Ds, is_3D = [], [], [], []
for i, mol in enumerate(suppl):
    pos = mol.GetConformer(0).GetPositions()
    is_3D.append(check_3d(pos))
    Zs.append([a.GetAtomicNum() for a in mol.GetAtoms()])
    euclid_Ds.append(Chem.Get3DDistanceMatrix(mol))
    graph_Ds.append(Chem.GetDistanceMatrix(mol))

In [ ]:
len(Zs), len(euclid_Ds), len(graph_Ds), len(is_3D)

In [ ]:
df['Z'] = Zs
df['euclid_D'] = euclid_Ds
df['graph_D']  = graph_Ds
df['is_3D'] = is_3D

In [ ]:
check_3d(suppl[3].GetConformer(0).GetPositions())

In [ ]:
print(suppl.GetItemText(3))

In [ ]:
df.to_json('data/sdf.json', orient='records', lines=True)

## Stuff

In [ ]:
def flatten(lst):
    return [item for slist in lst for item in slist]

In [ ]:
df = pd.read_json('data/sdf.json', orient='records', lines=True)

In [ ]:
Zs = flatten(df.Z.tolist())

In [ ]:
plt.hist(Zs)

In [ ]:
from collections import Counter
Counter(Zs)

In [ ]:
sizes = df.Z.apply(len)

In [ ]:
sizes.describe()

In [ ]:
df.is_3D.value_counts()

In [ ]:
euclid_D = flatten(flatten(df.euclid_D))
graph_D = flatten(flatten(df.graph_D))

In [ ]:
Counter(graph_D)

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(euclid_D, kde=False)

In [ ]:
idx = df.graph_D.apply(lambda x: len(np.intersect1d([100000000.0], x)) > 0)

In [ ]:
df[idx].is_3D

In [ ]:
suppl[32].GetConformer(0).GetPositions()

In [ ]:
check_3d(_)